## Potentially useful links
- https://towardsdatascience.com/monte-carlo-learning-b83f75233f92
- https://www.kth.se/social/files/58b941d5f276542843812288/RL04-Monte-Carlo.pdf
- https://medium.com/@zsalloum/q-vs-v-in-reinforcement-learning-the-easy-way-9350e1523031
- https://medium.com/@zsalloum/monte-carlo-in-reinforcement-learning-the-easy-way-564c53010511
- https://medium.com/@pedrohbtp/ai-monte-carlo-tree-search-mcts-49607046b204

In [154]:
import random
from random import randrange
import numpy as np
from Gridworld import Gridworld, DEFAULT

## Monte Carlo Algorithm

```
Intialise random policy with (s) = rand(a)
Initialise Q table with (s,a) = 0 
Initialise returns (s,a) = []

for N episodes
    state_action_returns = play_game(policy)
    
    for state_action_returns
        if not seen
            add state action to returns
            new Q value for (s a) = average (returns (s, a))
    
    for state in policy
        max action for each Q(state)
        policy for that state = Max action
```

In [157]:
def greed_action(action, state):
    if (random.random() > EPSILON):
        return action
    else:
        return random.choice([ action for action, reward in g.get_valid_moves(state) ])

def play_game(policy):
    g = Gridworld(DEFAULT)
    state_action_rewards = []
    
    state = (2,0)
    action = greed_action(policy[state], state)
    
    state_action_rewards.append((state, action, 0))

    
    while True:
        probs = g.transition_probabilities(action,state)
        
        # next_state 
        idx = np.random.choice(len(probs), p=[p for p, r, n_s in probs])
        _, reward, next_state = probs[idx]
        state = next_state
        
        
        if state in g.terminal_states:
            state_action_rewards.append((state, None, reward))
            break
        else:
            action = greed_action(policy[state], state)
            state_action_rewards.append((state, action, reward))
        
        state = next_state

    return state_action_rewards

def calculate_returns(state_action_rewards):
    '''
    Takes an array of (s[t-1], a[t-1], r)
    Returns an array of (s, a, G)
    '''
    
    state_action_returns = []
    
    state, action, reward = state_action_rewards.pop()
    G = reward
    
    for state, action, reward in reversed(state_action_rewards):
        state_action_returns.append((state, action , G))
        
        G = reward + GAMMA * G
    
    state_action_returns.reverse()
    return state_action_returns

In [ ]:
N_EPISODES = 1000
EPSILON = 0.2
GAMMA = 0.99


In [192]:

policy = {}
Q = {}
returns = {}
g = Gridworld(DEFAULT)

# Set up variables
for state in g.available_states:
    policy[state] = random.choice([ action for action, reward in g.get_valid_moves(state) ])
    
    Q[state] = {}
    for action, reward in g.get_valid_moves(state):
        Q[state][action] = 0
        returns[state, action] = []

# Run for x Episodes
for i in range(1000):
    state_action_rewards = play_game(policy)
    state_action_returns = calculate_returns(state_action_rewards)
    
    # Add new state action to returns
    for state, action, G in state_action_returns:
        returns[state, action].append(G)
    
    # Update Q table
    for state in g.available_states:
        for action, reward in g.get_valid_moves(state):
            if (len(returns[state, action]) > 0):
                Q[state][action] = sum(returns[state, action]) / len(returns[state, action])
    
    # Update policy
    for state in g.available_states:
        policy[state] = max(Q[state], key=Q[state].get)


In [193]:
Q

{(0, 0): {'UP': -0.5147121789103535,
  'RIGHT': 0.16527512783723738,
  'DOWN': -0.12284343844970244,
  'LEFT': -0.501316441956495},
 (0, 1): {'UP': -0.24572017086164472,
  'RIGHT': 0.48005673441673524,
  'LEFT': -0.0468073990350339},
 (0, 2): {'UP': -0.4651314378919191,
  'RIGHT': 0.6795883252569955,
  'LEFT': 0.3339049480603574},
 (0, 3): {'UP': 0.2844132689966502,
  'RIGHT': 0.882077726177756,
  'DOWN': 0.3090594929968574,
  'LEFT': 0.5133695358656334},
 (1, 0): {'UP': 0.053413383631128686,
  'DOWN': -0.48114131787488207,
  'LEFT': -0.7385887675980373},
 (1, 3): {'UP': 0.5558970839230097,
  'RIGHT': -0.9031763715625,
  'DOWN': -0.8737793282876933},
 (2, 0): {'UP': -0.1396153585062657,
  'RIGHT': -1.2522548360947725,
  'DOWN': -1.2141019626877751,
  'LEFT': -1.0670681424914923},
 (2, 1): {'RIGHT': -1.4758132298191304,
  'DOWN': -2.6382643765723524,
  'LEFT': -0.8801771238215549},
 (2, 2): {'RIGHT': -1.3094026287941996,
  'DOWN': -2.33418860550197,
  'LEFT': -1.509882733587003},
 (2, 3